1. hit rate at k
2. money_precision_at_k
3. recall_at_k
4. money_recall_at_k
5. map@k
6. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции
7. Mean Reciprocal Rank
8. *average_precision для ранжирования по примеру с картинки лекции

In [3]:
import numpy as np

In [1]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

**1. hit rate at k**

In [7]:
def hit_rate_k(bought_list, recommended_list, k=5):
    flags = np.isin(bought_list, recommended_list[:k])
    hit_rate = (flags.sum() > 0) * 1
    return hit_rate

In [8]:
hit_rate_k(bought_list, recommended_list)

1

**2. money_precision_at_k**

In [17]:
def money_precision_k(bought_list, recommended_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    money_precision = (flags * prices_recommended).sum() / prices_recommended.sum()
    
    return money_precision

In [11]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [400, 200, 300, 70, 100, 670, 320, 350, 420, 250]

In [27]:
money_precision_k(bought_list, recommended_list, prices_recommended, k=5)

0.4392523364485981

**3. recall_at_k**

In [28]:
def recall_k(bought_list, recommended_list, k=5):
    flags = np.isin(bought_list, recommended_list[:k])
    recall = flags.sum() / len(bought_list)
    return recall

In [29]:
recall_k(bought_list, recommended_list)

0.5

**4. money_recall_at_k**

In [65]:
def money_recall_k(recommended_list, bought_list, prices_bought, k=5):
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list[:k])
    flags = np.isin(bought_list, recommended_list)
    money_recall = (flags * prices_bought).sum() / prices_bought.sum() 
    return money_recall

In [66]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_bought = [260, 300, 400, 70]

In [67]:
money_recall_k(recommended_list, bought_list, prices_bought)

0.4563106796116505

**5. map@k**

In [40]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [41]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [243]:
ap_k(recommended_list, bought_list, k=5)

0.75

In [239]:
def map_k(recommended_lists, bought_lists, k=5):
    '''Ф-ция принимает 2 списка:
    recommended_lists - список со списками рекомендованных товаров пользователям; 
    bought_lists - список со списками купленных товаров пользователями
    '''
    sum_ = 0
    for recommended_list, bought_list in zip(recommended_lists, bought_lists):
        sum_ += ap_k(recommended_list, bought_list, k=k)
    result = sum_ / len(recommended_lists)
    return result

In [242]:
map_k([recommended_list], [bought_list], k=5)

0.75

**6. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции**

In [186]:
def DCG(ratings):
    ratings = np.array(ratings)
    return ratings[0] + (ratings[1:]/(np.log2(np.arange(3, len(ratings)+2)))).sum()


def NDCG_k(ratings, k=5):
    ratings = np.array(ratings[:k])
    
    # сортировка по убыванию, для расчета идеального DCG
    ideal_ratings = np.sort(ratings)[::-1]
    ideal_DCG = DCG(ideal_ratings)
    
    DCG_ = DCG(ratings)
    
    return DCG_ / ideal_DCG

In [188]:
ratings = [3, 2, 1, 1, 3, 1, 2]

NDCG_k(ratings, k=7)

0.9419493913323127

**7. Mean Reciprocal Rank**

In [214]:
def reciprocal_rank(recommended_list, bought_list):
    # индексы элементов, входящих в bought_list
    indices, = np.where(np.isin(recommended_list, bought_list))
    if indices.size == 0:
        return 0
    result = 1 / (indices[0] + 1)

    return result

In [215]:
reciprocal_rank(recommended_list, bought_list)

1.0

In [216]:
def MRR(recommended_lists, bought_lists):
    '''Ф-ция принимает 2 списка:
    recommended_lists - список со списками рекомендованных товаров пользователям; 
    bought_lists - список со списками купленных товаров пользователями
    '''
    ranks = [
        reciprocal_rank(recommended_list, bought_list) 
        for recommended_list, bought_list in zip(recommended_lists, bought_lists)]
    return np.mean(ranks)

In [218]:
MRR([recommended_list], [bought_list])

1.0

**8. *average_precision для ранжирования по примеру с картинки лекции**

In [237]:
def average_precision(recommended, relevants):
    relevants = np.array(relevants)
    recommended = np.array(recommended)

    flags = np.isin(recommended, relevants)
    
    # вместо цикла используется скалярное произведение
    ap = ((flags * np.cumsum(flags)) / (np.arange(len(flags)) + 1)).sum() / flags.sum()
    
    return ap

In [246]:
recommended = [6, 3, 5, 0, 4, 2, 1, 7]
relevants = [6, 5, 0, 2]

average_precision(recommended, relevants)

0.7708333333333333